In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from datasets import IMDBTrain, IMDBTest
from nlp_cls_trainer import Trainer, IteratorParams
from models import Transformer_Custom
from utils import loaders_from_dataset

In [ ]:
loaders = lambda batch_size, val_perc_size: loaders_from_dataset(IMDBTrain(), IMDBTest(), batch_size, val_perc_size)

params_clearml = {

}

DATASET_NAME = 'imdb_try'
params_trainer = {
    'model': Transformer_Custom,
    'loaders': loaders,
    'criterion': torch.nn.BCEWithLogitsLoss,
    'optim': torch.optim.AdamW,
    'scheduler': torch.optim.lr_scheduler.CosineAnnealingLR,
    'params_clearml': params_clearml,
    'is_tensorboard': True
}

trainer = Trainer(**params_trainer)

In [ ]:
%tensorboard --logdir=data

In [ ]:
vocab_size = len(IMDBTest().word2index)
model_ls = [{'vocab_size': vocab_size, 'emb_dim': 64}]
loaders_ls = [{'batch_size':32, 'val_perc_size': 0.0}]
criterion_ls = [{}]
optim_ls = [{'lr': 0.05, 'weight_decay': 0.001}]
scheduler_ls = [{'T_max':300}]

iter_params = IteratorParams(model_ls, loaders_ls, criterion_ls, optim_ls, scheduler_ls)

params_runs = {
    'iter_params': iter_params,
    'epochs': 100,
    'exp_name': f'{DATASET_NAME}',
    'val_step': 5,
    'verbose': False,
    'checkpoint_save_step': 25, 
    'device': device
}

trainer.run_trainer(**params_runs)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.imshow(trainer.model.pos_emb.detach().cpu())
plt.xlabel('Emb Dim')
plt.ylabel('Position')